In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#miRNA (HiSeq, Gene level)
file_name_list = [
          "drive/MyDrive/Copy of Human__TCGA_BRCA__BDGSC__miRNASeq__HS_miR__01_28_2016__BI__Gene__Firehose_RPKM_log2.cct",
          ]

id = 0
df = pd.DataFrame({})
for file_name in file_name_list:
    print(file_name)
    data_dict1 = {}
    # Read the file and populate the data dictionary
    with open(str(file_name), 'r') as file:
        for line in file:
            parts = line.strip().split()
            attribute_name = parts[0]
            attribute_values = parts[1:]
            data_dict1[attribute_name] = attribute_values

    # Create a DataFrame from the data dictionary

    if id == 0:
        df = pd.DataFrame(data_dict1)

    else:
        df2 = pd.DataFrame(data_dict1)
        df = df.merge(df2, on='attrib_name', how='inner')

    id += 1
df = df[df.iloc[:, 1:].isin(['NA']).any(axis=1) == False]
df

drive/MyDrive/Copy of Human__TCGA_BRCA__BDGSC__miRNASeq__HS_miR__01_28_2016__BI__Gene__Firehose_RPKM_log2.cct


,attrib_name,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,...,hsa-mir-940,hsa-mir-941-1,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
0,TCGA.3C.AAAU,13.1299,14.118,13.1479,14.5952,8.4191,8.6695,10.5228,3.9742,11.8252,...,2.8825,0,2.7072,0.2479,1.6146,2.3574,6.9187,5.7812,7.0356,15.5065
1,TCGA.3C.AALI,12.9183,13.9224,12.9134,14.5127,9.6483,9.0065,9.1343,4.4543,12.6791,...,1.0712,0,3.1985,0,0.7042,1.1753,5.9366,6.4437,7.8914,13.6263
2,TCGA.3C.AALJ,13.0122,14.0101,13.0287,13.4197,9.3147,9.2793,11.3962,5.3505,13.5304,...,1.2114,0,2.2073,0,0.8217,1.4031,6.6184,6.8905,7.5882,15.0139
3,TCGA.3C.AALK,13.1449,14.1418,13.1514,14.6673,11.5119,8.3891,10.3701,4.2378,12.6528,...,0.9625,0,2.163,0,2.163,1.4538,6.2011,5.4122,10.033,14.5548
4,TCGA.4H.AAAK,13.4118,14.4136,13.4206,14.4386,11.6944,8.4579,10.7422,4.5572,13.0097,...,0.4053,0,1.6361,0,1.0944,0.8567,4.3887,5.1449,10.0795,14.6504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,TCGA.WT.AB44,13.3759,14.3667,13.37,14.5141,11.9267,8.5241,10.5369,4.609,13.3679,...,0.8573,0,1.1489,0,0.4914,1.1489,6.3365,5.8735,10.1058,13.9961
751,TCGA.XX.A899,14.0362,15.0364,14.0434,14.3396,12.362,7.663,11.0863,5.2437,14.1539,...,0.5887,0,2.6649,0,2.0442,1.7304,4.14,5.8666,10.9073,13.7815
752,TCGA.XX.A89A,13.6797,14.6849,13.6916,14.1983,12.6844,8.5569,11.3152,4.8829,14.0395,...,1.4205,0,1.5176,0,0.6406,0.9498,5.4236,5.787,10.3566,14.0595
753,TCGA.Z7.A8R5,12.9623,13.9664,12.9851,14.3207,11.9806,8.2123,9.9116,4.6372,12.8967,...,0.9756,0,1.6187,0,1.6821,1.4097,5.8132,5.7553,10.1318,14.1906


In [ ]:
#row = df[df['attrib_name'] == 'TCGA.A2.A04P']

# Print the values of columns "C5orf40_y" and "C5orf40_x" for the selected row
#print("C5orf40_y:", row['C5orf40_y'].values[0])
#print("C5orf40_x:", row['C5orf40_x'].values[0])

In [ ]:
#Separate text descriptions from numeric attributes
text_descriptions = df.iloc[:,0]
numeric_attributes = df.iloc[:,1:]
print(numeric_attributes)
print(numeric_attributes.shape[1])

    hsa-let-7a-1 hsa-let-7a-2 hsa-let-7a-3 hsa-let-7b hsa-let-7c hsa-let-7d  \
0        13.1299       14.118      13.1479    14.5952     8.4191     8.6695   
1        12.9183      13.9224      12.9134    14.5127     9.6483     9.0065   
2        13.0122      14.0101      13.0287    13.4197     9.3147     9.2793   
3        13.1449      14.1418      13.1514    14.6673    11.5119     8.3891   
4        13.4118      14.4136      13.4206    14.4386    11.6944     8.4579   
..           ...          ...          ...        ...        ...        ...   
750      13.3759      14.3667        13.37    14.5141    11.9267     8.5241   
751      14.0362      15.0364      14.0434    14.3396     12.362      7.663   
752      13.6797      14.6849      13.6916    14.1983    12.6844     8.5569   
753      12.9623      13.9664      12.9851    14.3207    11.9806     8.2123   
754      13.3498        14.35      13.3786    14.1173    10.3791     9.6291   

    hsa-let-7e hsa-let-7f-1 hsa-let-7f-2 hsa-let-7g

In [ ]:
!pip install keras-tuner --upgrade


In [ ]:
#https://www.analyticsvidhya.com/blog/2021/06/dimensionality-reduction-using-autoencoders-in-python/
import math
import pandas as pd
import tensorflow as tf
import kerastuner.tuners as kt
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import numpy as np


# Train the autoencoder with the training data.
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),#
          Dense(256, activation="relu"),
          Dense(128, activation="relu"),

        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(512, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded


def create_and_train_autoencoder(x_train_scaled,x_test_scaled):
    auto_encoder = AutoEncoders(len(x_train_scaled.columns))
    auto_encoder.compile(
        loss='mae',
        metrics=['mae'],
        #optimizer='adam'
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# 0.001 to avoid overlearning
    )
    history = auto_encoder.fit(
        x_train_scaled,
        x_train_scaled,
        epochs=50,
        batch_size=32,
        validation_data=(x_test_scaled, x_test_scaled),
        verbose=0
    )
    return history




In [ ]:
# Split the data
standard_scaler = MinMaxScaler()
numeric_attributes_scaled = pd.DataFrame(standard_scaler.fit_transform(numeric_attributes),columns=numeric_attributes.columns)

In [ ]:
x_train_scaled, x_test_scaled = train_test_split(numeric_attributes_scaled, test_size=0.3, random_state=42)

auto_encoder = AutoEncoders(len(x_train_scaled.columns))
auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    #optimizer='adam'
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# 0.001 to avoid overlearning
)
history = auto_encoder.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=50,
    batch_size=32,
    validation_data=(x_test_scaled, x_test_scaled),
    #verbose=0
)
encoder_layer = auto_encoder.layers[0]  # Get the encoder layer
X_encoded = pd.DataFrame(encoder_layer.predict(numeric_attributes_scaled))  # Reduced_df
#X_encoded

Epoch 1/50
17/17 [==============================] - 1s 27ms/step - loss: 0.1733 - mae: 0.1733 - val_loss: 0.0953 - val_mae: 0.0953
Epoch 2/50
17/17 [==============================] - 0s 15ms/step - loss: 0.0900 - mae: 0.0900 - val_loss: 0.0852 - val_mae: 0.0852
Epoch 3/50
17/17 [==============================] - 0s 18ms/step - loss: 0.0841 - mae: 0.0841 - val_loss: 0.0820 - val_mae: 0.0820
Epoch 4/50
17/17 [==============================] - 0s 27ms/step - loss: 0.0810 - mae: 0.0810 - val_loss: 0.0788 - val_mae: 0.0788
Epoch 5/50
17/17 [==============================] - 0s 22ms/step - loss: 0.0786 - mae: 0.0786 - val_loss: 0.0757 - val_mae: 0.0757
Epoch 6/50
17/17 [==============================] - 0s 18ms/step - loss: 0.0760 - mae: 0.0760 - val_loss: 0.0737 - val_mae: 0.0737
Epoch 7/50
17/17 [==============================] - 0s 17ms/step - loss: 0.0744 - mae: 0.0744 - val_loss: 0.0725 - val_mae: 0.0725
Epoch 8/50
17/17 [==============================] - 0s 17ms/step - loss: 0.0728 - m

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.000000,0.0,1.910987,0.0,0.0,0.0,0.0,1.653529,0.000000,1.842533,...,0.622166,0.038883,0.0,0.496235,0.0,0.0,0.0,0.0,0.953027,0.0
1,0.791876,0.0,2.148519,0.0,0.0,0.0,0.0,2.091879,0.000000,0.564291,...,0.706692,1.233945,0.0,0.190356,0.0,0.0,0.0,0.0,1.737383,0.0
2,0.862084,0.0,1.415663,0.0,0.0,0.0,0.0,1.380910,0.000000,1.138549,...,0.381248,0.403755,0.0,0.000000,0.0,0.0,0.0,0.0,2.034071,0.0
3,1.783058,0.0,1.217995,0.0,0.0,0.0,0.0,1.110916,0.495788,0.919777,...,0.396657,0.636511,0.0,0.000000,0.0,0.0,0.0,0.0,2.179386,0.0
4,2.523397,0.0,1.197713,0.0,0.0,0.0,0.0,0.878679,0.747193,0.926881,...,0.620262,0.097861,0.0,0.000000,0.0,0.0,0.0,0.0,2.715928,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,2.021744,0.0,1.371343,0.0,0.0,0.0,0.0,1.022864,0.324529,0.633003,...,0.139473,0.303144,0.0,0.000000,0.0,0.0,0.0,0.0,2.280053,0.0
751,2.045001,0.0,0.738993,0.0,0.0,0.0,0.0,0.562660,0.000000,0.673281,...,0.830286,1.039893,0.0,0.575704,0.0,0.0,0.0,0.0,2.063425,0.0
752,2.172485,0.0,0.601103,0.0,0.0,0.0,0.0,0.952510,0.183267,0.162008,...,0.565569,1.464523,0.0,0.002846,0.0,0.0,0.0,0.0,1.766279,0.0
753,1.889453,0.0,1.165708,0.0,0.0,0.0,0.0,0.433680,0.378354,0.685851,...,0.159072,0.709140,0.0,0.475601,0.0,0.0,0.0,0.0,2.046999,0.0


In [ ]:
#Epoch 50/50
#17/17 [==============================] - 0s 22ms/step - loss: 0.0539 - mae: 0.0539 - val_loss: 0.0616 - val_mae: 0.0616
#24/24 [==============================] - 0s 4ms/step


In [ ]:
X_encoded

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.000000,0.0,1.910987,0.0,0.0,0.0,0.0,1.653529,0.000000,1.842533,...,0.622166,0.038883,0.0,0.496235,0.0,0.0,0.0,0.0,0.953027,0.0
1,0.791876,0.0,2.148519,0.0,0.0,0.0,0.0,2.091879,0.000000,0.564291,...,0.706692,1.233945,0.0,0.190356,0.0,0.0,0.0,0.0,1.737383,0.0
2,0.862084,0.0,1.415663,0.0,0.0,0.0,0.0,1.380910,0.000000,1.138549,...,0.381248,0.403755,0.0,0.000000,0.0,0.0,0.0,0.0,2.034071,0.0
3,1.783058,0.0,1.217995,0.0,0.0,0.0,0.0,1.110916,0.495788,0.919777,...,0.396657,0.636511,0.0,0.000000,0.0,0.0,0.0,0.0,2.179386,0.0
4,2.523397,0.0,1.197713,0.0,0.0,0.0,0.0,0.878679,0.747193,0.926881,...,0.620262,0.097861,0.0,0.000000,0.0,0.0,0.0,0.0,2.715928,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,2.021744,0.0,1.371343,0.0,0.0,0.0,0.0,1.022864,0.324529,0.633003,...,0.139473,0.303144,0.0,0.000000,0.0,0.0,0.0,0.0,2.280053,0.0
751,2.045001,0.0,0.738993,0.0,0.0,0.0,0.0,0.562660,0.000000,0.673281,...,0.830286,1.039893,0.0,0.575704,0.0,0.0,0.0,0.0,2.063425,0.0
752,2.172485,0.0,0.601103,0.0,0.0,0.0,0.0,0.952510,0.183267,0.162008,...,0.565569,1.464523,0.0,0.002846,0.0,0.0,0.0,0.0,1.766279,0.0
753,1.889453,0.0,1.165708,0.0,0.0,0.0,0.0,0.433680,0.378354,0.685851,...,0.159072,0.709140,0.0,0.475601,0.0,0.0,0.0,0.0,2.046999,0.0


In [ ]:
text_descriptions

0      TCGA.3C.AAAU
1      TCGA.3C.AALI
2      TCGA.3C.AALJ
3      TCGA.3C.AALK
4      TCGA.4H.AAAK
           ...     
750    TCGA.WT.AB44
751    TCGA.XX.A899
752    TCGA.XX.A89A
753    TCGA.Z7.A8R5
754    TCGA.Z7.A8R6
Name: attrib_name, Length: 755, dtype: object

In [ ]:
X_encoded.insert(0, 'attrib_name', text_descriptions)
X_encoded

,attrib_name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,TCGA.3C.AAAU,0.000000,0.0,1.910987,0.0,0.0,0.0,0.0,1.653529,0.000000,...,0.622166,0.038883,0.0,0.496235,0.0,0.0,0.0,0.0,0.953027,0.0
1,TCGA.3C.AALI,0.791876,0.0,2.148519,0.0,0.0,0.0,0.0,2.091879,0.000000,...,0.706692,1.233945,0.0,0.190356,0.0,0.0,0.0,0.0,1.737383,0.0
2,TCGA.3C.AALJ,0.862084,0.0,1.415663,0.0,0.0,0.0,0.0,1.380910,0.000000,...,0.381248,0.403755,0.0,0.000000,0.0,0.0,0.0,0.0,2.034071,0.0
3,TCGA.3C.AALK,1.783058,0.0,1.217995,0.0,0.0,0.0,0.0,1.110916,0.495788,...,0.396657,0.636511,0.0,0.000000,0.0,0.0,0.0,0.0,2.179386,0.0
4,TCGA.4H.AAAK,2.523397,0.0,1.197713,0.0,0.0,0.0,0.0,0.878679,0.747193,...,0.620262,0.097861,0.0,0.000000,0.0,0.0,0.0,0.0,2.715928,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,TCGA.WT.AB44,2.021744,0.0,1.371343,0.0,0.0,0.0,0.0,1.022864,0.324529,...,0.139473,0.303144,0.0,0.000000,0.0,0.0,0.0,0.0,2.280053,0.0
751,TCGA.XX.A899,2.045001,0.0,0.738993,0.0,0.0,0.0,0.0,0.562660,0.000000,...,0.830286,1.039893,0.0,0.575704,0.0,0.0,0.0,0.0,2.063425,0.0
752,TCGA.XX.A89A,2.172485,0.0,0.601103,0.0,0.0,0.0,0.0,0.952510,0.183267,...,0.565569,1.464523,0.0,0.002846,0.0,0.0,0.0,0.0,1.766279,0.0
753,TCGA.Z7.A8R5,1.889453,0.0,1.165708,0.0,0.0,0.0,0.0,0.433680,0.378354,...,0.159072,0.709140,0.0,0.475601,0.0,0.0,0.0,0.0,2.046999,0.0


In [ ]:
X_encoded.to_csv("drive/MyDrive/miRNASeq__HS_miR_128.csv", index=False)